In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 897.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
!pip install faiss-gpu langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.4 MB/s eta 0:00:00


In [4]:
import os
import json
import re
from typing import List, Type, TypeVar
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
import faiss
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Define a generic type variable bound to BaseModel
T = TypeVar('T', bound=BaseModel)

def extract_multi_needle(schema: Type[T], haystack: str, example_needles: List[str]) -> List[T]:
    """
    Extracts and structures information from a large text corpus based on a given schema and examples.

    Args:
        schema (Type[T]): A Pydantic model defining the structure of the needle to be extracted.
        haystack (str): The large text corpus to search through (haystack).
        example_needles (List[str]): A list of example sentences (needles).

    Returns:
        List[T]: A list of extracted needles conforming to the provided schema.
    """
    # Initialize the list to hold the extracted needles
    extracted_needles = []

    # Initialize the SentenceTransformer model for embeddings
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient model for embeddings

    # Split the haystack into sentences using regex
    sentences = re.split(r'(?<=[.!?])\s+', haystack)

    # Compute embeddings for the sentences in the haystack
    sentence_embeddings = embedding_model.encode(sentences, batch_size=256, show_progress_bar=True)

    # Compute embeddings for the example needles
    example_embeddings = embedding_model.encode(example_needles, show_progress_bar=True)

    # Build a Faiss index for efficient similarity search
    dimension = sentence_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(sentence_embeddings)
    index.add(sentence_embeddings)

    # Normalize example embeddings for cosine similarity
    faiss.normalize_L2(example_embeddings)

    # Number of nearest neighbors to retrieve
    k = 100

    # Perform similarity search for each example embedding
    D, I = index.search(example_embeddings, k)

    # Collect candidate sentences based on similarity search
    candidate_sentences = set()
    for indices in I:
        for idx in indices:
            candidate_sentences.add(sentences[idx])

    # Convert the set to a list for processing
    candidate_sentences = list(candidate_sentences)

    # Initialize the Azure OpenAI LLM model
    model = AzureChatOpenAI(
        openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-03-15-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
        azure_endpoint=os.environ.get(
            "AZURE_OPENAI_ENDPOINT",
            "https://gptmini4o.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2023-03-15-preview"
        ),
        openai_api_key=os.environ.get("AZURE_OPENAI_KEY", "c2105be0c2744742980b57320b87e813"),
    )

    # Generate a description of the schema fields and their descriptions
    def generate_schema_description(schema: Type[BaseModel]) -> str:
        """
        Generates a string description of the schema.

        Args:
            schema (Type[BaseModel]): The Pydantic model.

        Returns:
            str: A string describing the schema fields and their descriptions.
        """
        schema_description = ""
        for field_name, field in schema.__fields__.items():
            field_desc = field.description or ''
            field_type = (
            field.annotation.__name__ if hasattr(field.annotation, '__name__') else str(field.annotation)
        )
            schema_description += f"- {field_name} ({field_type}): {field_desc}\n"
        return schema_description

    schema_description = generate_schema_description(schema)

    # Construct the system prompt with schema description
    system_prompt = f"""
You are an assistant that extracts information from text according to a given schema.

The schema is:
{schema_description}

Your task is to read the provided text and extract any information that matches the schema.

Provide the extracted data as a JSON object conforming to the schema.

If the text does not contain relevant information, output an empty JSON object.

Only provide the JSON object, and no additional text.
"""

    # Process each candidate sentence
    for text in candidate_sentences:
        # Create the conversation messages for the LLM
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=text)
        ]

        # Call the LLM to process the text
        response = model(messages)

        # Attempt to parse the LLM response as JSON
        try:
            data = json.loads(response.content)
            if data:  # If data is not empty
                # Validate and instantiate the schema
                item = schema(**data)
                extracted_needles.append(item)
        except json.JSONDecodeError:
            # If parsing fails, skip this text
            continue
        except Exception:
            # If data does not conform to schema, skip
            continue

    return extracted_needles


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
from typing import Optional
from pydantic import BaseModel, Field

class TechCompany(BaseModel):
    name: Optional[str] = Field(default=None, description="The full name of the technology company")
    location: Optional[str] = Field(default=None, description="City and country where the company is headquartered")
    employee_count: Optional[int] = Field(default=None, description="Total number of employees")
    founding_year: Optional[int] = Field(default=None, description="Year the company was established")
    is_public: Optional[bool] = Field(default=None, description="Whether the company is publicly traded (True) or privately held (False)")
    valuation: Optional[float] = Field(default=None, description="Company's valuation in billions of dollars")
    primary_focus: Optional[str] = Field(default=None, description="Main area of technology or industry the company focuses on")

In [6]:
example_needles = ["Ryoshi, based in Neo Tokyo, Japan, is a private quantum computing firm founded in 2031, currently valued at $8.7 billion with 1,200 employees focused on quantum cryptography."]

In [9]:
with open("haystack.txt", "r") as file:
    haystack_text = file.read()

In [10]:
# Example usage
extracted_data = extract_multi_needle(schema=TechCompany, haystack=haystack_text, example_needles=example_needles)

# Serialize the extracted data to a JSON file
with open('extracted_needles.json', 'w') as f:
    json.dump([item.dict() for item in extracted_data], f, indent=2)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1070 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-4-272d0a5ef333>:123: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model(messages)


In [11]:
model = AzureChatOpenAI(
        openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-03-15-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
        azure_endpoint=os.environ.get(
            "AZURE_OPENAI_ENDPOINT",
            "https://gptmini4o.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2023-03-15-preview"
        ),
        openai_api_key=os.environ.get("AZURE_OPENAI_KEY", "c2105be0c2744742980b57320b87e813"),
    )

In [12]:
model.invoke("hey").content

'Hey! How can I help you today?'

In [ ]:
from typing import List, Type, TypeVar, Optional, Dict, Any
from pydantic import BaseModel
import spacy
import openai
import json
import re
import numpy as np
from sentence_transformers import SentenceTransformer, util

T = TypeVar('T', bound=BaseModel)

def extract_multi_needle(schema: Type[T], haystack: str, example_needles: List[str]) -> List[T]:
    """
    Extracts and structures information from a large text corpus based on a given schema and examples.

    Args:
    schema (Type[T]): A Pydantic model defining the structure of the needle to be extracted.
    haystack (str): The large text corpus to search through (haystack).
    example_needles (List[str]): A list of example sentences (needles).

    Returns:
    List[T]: A list of extracted needles conforming to the provided schema.
    """
    extracted_needles = []

    # Initialize spaCy model
    nlp = spacy.load('en_core_web_sm')
    nlp.max_length = len(haystack) + 1000  # Adjust based on the input size

    # Step 1: Extract keywords from example_needles
    keywords = extract_keywords(example_needles, nlp)

    # Step 2: Preprocess haystack and split into sentences
    haystack_sentences = split_into_sentences(haystack, nlp)

    # Step 3: Find candidate sentences using keyword matching
    candidate_sentences = find_candidate_sentences(haystack_sentences, keywords)

    if not candidate_sentences:
        return extracted_needles  # No candidates found

    # Step 4: Use embeddings to compute similarity between examples and candidates
    num_candidates = 100  # Adjust based on desired processing time
    candidate_sentences = rank_candidates_by_similarity(example_needles, candidate_sentences, num_candidates)

    # Step 5: Extract data from candidate sentences using LLM
    for sentence in candidate_sentences:
        prompt = construct_prompt(schema, sentence)
        response = call_llm_api(prompt)
        data = parse_llm_response(response)
        try:
            instance = schema.parse_obj(data)
            extracted_needles.append(instance)
        except Exception:
            # Handle validation errors silently or log them
            pass

    return extracted_needles

def extract_keywords(example_needles: List[str], nlp) -> set:
    combined_text = ' '.join(example_needles)
    doc = nlp(combined_text)
    keywords = {token.lemma_.lower() for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ'}}
    return keywords

def split_into_sentences(text: str, nlp) -> List[str]:
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

def find_candidate_sentences(sentences: List[str], keywords: set) -> List[str]:
    candidate_sentences = []
    for sentence in sentences:
        sentence_lower = sentence.lower()
        if any(keyword in sentence_lower for keyword in keywords):
            candidate_sentences.append(sentence)
    return candidate_sentences

def rank_candidates_by_similarity(example_needles: List[str], candidate_sentences: List[str], num_candidates: int) -> List[str]:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    example_embeddings = model.encode(example_needles, convert_to_tensor=True)
    candidate_embeddings = model.encode(candidate_sentences, convert_to_tensor=True)
    cosine_scores = util.cos_sim(example_embeddings, candidate_embeddings)
    max_similarities = cosine_scores.max(axis=0).values
    top_indices = np.argsort(-max_similarities.cpu().numpy())[:num_candidates]
    top_candidate_sentences = [candidate_sentences[idx] for idx in top_indices]
    return top_candidate_sentences

def construct_prompt(schema: Type[T], sentence: str) -> str:
    field_info = []
    for field_name, field in schema.__fields__.items():
        description = field.description or ''
        field_type = (
            field.annotation.__name__ if hasattr(field.annotation, '__name__') else str(field.annotation)
        )
        field_info.append(f"- {field_name} ({field_type}): {description}")
    field_info_text = '\n'.join(field_info)
    prompt = f"""Extract the following information from the sentence:

Sentence: "{sentence}"

Information to extract:
{field_info_text}

Return the information as a JSON object with keys matching the field names.
If a piece of information is not available, use null.

Example format:
{{
    "field1": value1,
    "field2": value2,
    ...
}}
"""
    return prompt

def call_llm_api(prompt: str) -> str:
    # Replace 'YOUR_API_KEY' with your actual OpenAI API key
    model = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2024-07-18"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
    azure_endpoint=os.environ.get(
        "AZURE_OPENAI_ENDPOINT",
        "https://gptmini4o.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2023-03-15-preview"
    ),
    openai_api_key=os.environ.get("AZURE_OPENAI_KEY", "your_default_api_key_here"),
    )
    response = model.invoke(
        input=[
            {'role': 'system', 'content': 'You are an assistant that extracts structured information from text.'},
            {'role': 'user', 'content': prompt}
        ],
        temperature=0,
    )
    return response.content

def parse_llm_response(response: str) -> Dict[str, Any]:
    try:
        data = json.loads(response)
        return data
    except json.JSONDecodeError:
        match = re.search(r'\{.*\}', response, re.DOTALL)
        if match:
            json_str = match.group(0)
            try:
                data = json.loads(json_str)
                return data
            except json.JSONDecodeError:
                return {}
        else:
            return {}

